# day 5

https://adventofcode.com/2019/day/5

In [ ]:
import os

import eri.logging as logging

In [ ]:
FNAME = os.path.join('data', 'day05.txt')

LOGGER = logging.getLogger('day05')
logging.configure()

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = [1002, 4, 3, 4, 33]

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return [int(_) for _ in fp.read().strip().split(',')]

#### function def

In [ ]:
def parse_opcode(x):
    x5 = '{:0>5}'.format(x)
    return (int(x5[3:]),
            int(x5[2]),
            int(x5[1]),
            int(x5[0]))
#     return (x % 10,
#             x // 10_000,
#             x // 1_000)

In [ ]:
assert parse_opcode(1002) == (2, 0, 1, 0)

In [ ]:
POSITION = 0
IMMEDIATE = 1

In [ ]:
def lookup_with_mode(i, param_mode, data):
    # get the value in data
    try:
        val = data[i]
    except IndexError:
        return None, None
    
    try:
        x = val if param_mode == IMMEDIATE else data[val]
        return val, x
    except IndexError:
        return val, None

In [ ]:
def intcode_compute(data, inp=1, non_zero_only_on_halt=True):
    inst_ptr = 0
    while True:
        LOGGER.debug(f'data = {data}')
        LOGGER.debug(f'inst_ptr = {inst_ptr}')
        LOGGER.info(f'data[inst_ptr: inst_ptr + 4] = {data[inst_ptr: inst_ptr + 4]}')
        
        opcode, param_a, param_b, param_c = parse_opcode(data[inst_ptr])
        
        if opcode == 99:
            return 0
        
        a_val, a = lookup_with_mode(inst_ptr + 1, param_a, data)
        b_val, b = lookup_with_mode(inst_ptr + 2, param_b, data)
        c_val, c = lookup_with_mode(inst_ptr + 3, param_c, data)
        
        LOGGER.debug(f'opcode = {opcode}')
        LOGGER.debug(f"a_val = {a_val}")
        LOGGER.debug(f"a (in {'immediate' if param_a == IMMEDIATE else 'position'} mode) = {a}")
        LOGGER.debug(f"b_val = {b_val}")
        LOGGER.debug(f"b (in {'immediate' if param_b == IMMEDIATE else 'position'} mode) = {b}")
        LOGGER.debug(f"c_val = {c_val}")
        LOGGER.debug(f"c (in {'immediate' if param_c == IMMEDIATE else 'position'} mode) = {c}")
        
        if opcode == 1:
            LOGGER.debug('adding')
            data[c_val] = a + b
            inst_ptr += 4
        elif opcode == 2:
            LOGGER.debug('multiplying')
            data[c_val] = a * b
            inst_ptr += 4
        elif opcode == 3:
            LOGGER.debug('reading input')
            data[a_val] = inp
            inst_ptr += 2
        elif opcode == 4:
            LOGGER.debug('outputting')
            output = a
            if output != 0:
                LOGGER.debug(f'b_val = {b_val}')
                # check if the next element is a halt
                if non_zero_only_on_halt and (b_val % 100 != 99):
                    msg = f'error output = {output}'
                    LOGGER.error(msg)
                    raise ValueError(msg)
                else:
                    return output
            else:
                LOGGER.debug('test passed')
            inst_ptr += 2
        elif opcode == 5:
            LOGGER.debug('jump-if-true (aka non-zero)')
            if a != 0:
                inst_ptr = b
            else:
                inst_ptr += 3
        elif opcode == 6:
            LOGGER.debug('jump-if-false (aka zero)')
            if a == 0:
                inst_ptr = b
            else:
                inst_ptr += 3
        elif opcode == 7:
            LOGGER.debug('less than')
            data[c_val] = int(a < b)
            inst_ptr += 4
        elif opcode == 8:
            LOGGER.debug('equal')
            data[c_val] = int(a == b)
            inst_ptr += 4
        else:
            raise ValueError(f'opcode = {opcode}')

In [ ]:
intcode_compute([1002, 4, 3, 4, 33])

In [ ]:
def q_1(data):
    return intcode_compute(data)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    for inp in [1, 10, 100, 1000]:
        assert intcode_compute([3, 0, 4, 0, 99], inp=inp) == inp
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
LOGGER.setLevel(logging.WARNING)
x = q_1(load_data())
LOGGER.setLevel(logging.INFO)
x

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data, inp=1, non_zero_only_on_halt=True):
    LOGGER.warning('starting q_2')
    LOGGER.debug(f'data = {data}')
    LOGGER.debug(f'inp = {inp}')
    return intcode_compute(data, inp, non_zero_only_on_halt)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    # input == 8
    assert q_2([3,9,8,9,10,9,4,9,99,-1,8], 1) == 0
    assert q_2([3,9,8,9,10,9,4,9,99,-1,8], 8) == 1
    
    # input < 8
    assert q_2([3,9,7,9,10,9,4,9,99,-1,8], 8) == 0
    assert q_2([3,9,7,9,10,9,4,9,99,-1,8], 7) == 1
    
    # input == 8
    assert q_2([3,3,1108,-1,8,3,4,3,99], 1) == 0
    assert q_2([3,3,1108,-1,8,3,4,3,99], 8) == 1
    
    # input < 8
    assert q_2([3,3,1107,-1,8,3,4,3,99], 8) == 0
    assert q_2([3,3,1107,-1,8,3,4,3,99], 7) == 1
    
    # 0 if 0, 1 else
    assert q_2([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], 0) == 0
    assert q_2([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], 8) == 1
    assert q_2([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], 0) == 0
    assert q_2([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], 8) == 1

    big_prog = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
                1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
                999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    assert q_2(big_prog, 7, False) == 999
    assert q_2(big_prog, 8, False) == 1000
    assert q_2(big_prog, 9, False) == 1001
    
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data(), 5)

fin